In [1]:
import os
import numpy as np
import lmfit
import csv
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

In [2]:
# helper functions

def sinusoid(x, a, b, c):
    return a * np.sin(b * x + c)

def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]

def regress_sine_wave(x, y):
    model = lmfit.models.SineModel()
    y = savgol_filter(y, window_length=11, polyorder=3)
    y -= np.mean(y)
    y /= (max(y) - min(y))
    params = model.guess(y, x)
    fit = model.fit(y, params, x=x)

    lmfit_amp = fit.params['amplitude'].value
    lmfit_omega = fit.params['frequency'].value
    lmfit_shift = fit.params['shift'].value

    return [lmfit_amp, lmfit_omega, lmfit_shift]

def get_period_and_offset(params):
    amp = params[0]
    omega = params[1]
    shift = params[2]
    period = 2 * (np.pi) / omega

    offset = (0.5 * np.pi / float(omega)) - (float(shift) / float(omega))

    if (amp < 0.0):
        offset -= (np.pi) / (float(omega))

    while (offset < 0.0): 
        offset += period

    while (offset > period):
        offset -= period

    return [period, offset]

In [3]:
''' LAYOUT OF TEMPORAL OPTIMIZATION PIPELINE

define lists for offset and period

[1] find correct sides, offsets, and periods

for each scale normalized series:

    load the series

    regress left and right heel_y's, storing left and right params

    save period as 2pi / omega
    save offset as (0.5 * pi / omega) - (shift/omega)
        add period until offset is greater than 0
        subtract period until offset less than 2pi
    choose side with lower offset

[2] trim examples

find minimum period

for each scale normalized file:
    if foot and entryside mismatch:
        swap all L-R coords (helper function)

    find number of examples: min (((total frames - offset) / period), 3)
    find step size: current_period / min_period

    define a new series and step through at the step size, picking values to put in the normalized list

    for each number of examples:
        define a new series and append all the stepped frame values 

'''

" LAYOUT OF TEMPORAL OPTIMIZATION PIPELINE\n\ndefine lists for offset and period\n\n[1] find correct sides, offsets, and periods\n\nfor each scale normalized series:\n\n    load the series\n\n    regress left and right heel_y's, storing left and right params\n\n    save period as 2pi / omega\n    save offset as (0.5 * pi / omega) - (shift/omega)\n        add period until offset is greater than 0\n        subtract period until offset less than 2pi\n    choose side with lower offset\n\n[2] trim examples\n\nfind minimum period\n\nfor each scale normalized file:\n    if foot and entryside mismatch:\n        swap all L-R coords (helper function)\n\n    find number of examples: min (((total frames - offset) / period), 3)\n    find step size: current_period / min_period\n\n    define a new series and step through at the step size, picking values to put in the normalized list\n\n    for each number of examples:\n        define a new series and append all the stepped frame values \n\n"

In [6]:
# [1] Finding periods and offsets

scale_normalized_dir = '..\\time_series\\scale_normalized\\'
scale_normalized_files = os.listdir(scale_normalized_dir)

periods = {}
offsets = {}
sides = {}

for file in scale_normalized_files:
    series, num_frames = load_series(scale_normalized_dir + file)

    X = np.arange(0, num_frames)

    l_params = regress_sine_wave(X, series['left_heel_y'])
    r_params = regress_sine_wave(X, series['right_heel_y'])

    l_period_offset = get_period_and_offset(l_params)
    r_period_offset = get_period_and_offset(r_params)


    '''
    PLOTTING

    fig, ax = plt.subplots(2, figsize=(20, 10))
    ax[0].plot(X, series['left_heel_y'])
    ax[0].plot(X, sinusoid(X, *l_params))
    ax[0].axvline(l_period_offset[1], c='r')
    ax[0].axvline(np.sum(l_period_offset), c='b')

    ax[1].plot(X, series['right_heel_y'])
    ax[1].plot(X, sinusoid(X, *r_params))
    ax[1].axvline(r_period_offset[1], c='r')
    ax[1].axvline(np.sum(r_period_offset), c='b') 
    
    '''

    if (l_period_offset[1] < r_period_offset[1]): # if the left offset is smaller
        periods[file] = l_period_offset[0]
        offsets[file] = l_period_offset[1]
        sides[file] = 'L'
    else:
        periods[file] = r_period_offset[0]
        offsets[file] = r_period_offset[1]
        sides[file] = 'R'

print(periods)
print(offsets)
print(sides)

{'SR-O0-F0-B2-S7-L1-R-0059.csv': 21.895829527165585, 'SR-O0-F0-B6-S4-L7-R-0051.csv': 22.399868888307008, 'SR-O0-F3-B0-S1-L8-R-0036.csv': 22.349622666426608, 'SR-O0-F4-B0-S2-L2-R-0007.csv': 21.290065932242534, 'SR-O0-F4-B0-S2-L2-R-0009.csv': 20.68372881683032, 'SR-O0-F4-B0-S4-L7-R-0003.csv': 20.90333442270948, 'SR-O0-F5-B0-S2-L8-L-0002.csv': 22.249412333940672, 'SR-O0-F6-B0-S1-L7-R-0034.csv': 24.27532206357557, 'SR-O0-F6-B0-S2-L8-L-0033.csv': 24.885233754238854, 'SR-O0-F6-B0-S2-L9-L-0008.csv': 20.881344462791244, 'SR-O0-F8-B0-S4-L3-R-0001.csv': 20.745708596251117, 'SR-O0-F9-B0-S4-L3-R-0005.csv': 20.28936521231477, 'SR-O1-F0-B5-S5-L7-L-0050.csv': 11.15759453623121, 'SR-O2-F0-B3-S3-L2-L-0026.csv': 24.796081300862983, 'SR-O2-F0-B5-S3-L1-R-0057.csv': 22.20316934862861, 'SR-O2-F3-B0-S3-L5-R-0072.csv': 23.9121092723036, 'SR-O2-F5-B0-S3-L7-R-0063.csv': 23.13023179017117, 'SR-O3-F0-B5-S1-L4-L-0054.csv': 23.430862976298783, 'SR-O3-F0-B7-S2-L9-L-0024.csv': 24.62501642515268, 'SR-O3-F0-B8-S2-L5-R-